<h1> Refactor to support out-of-memory data and custom features</h1>

In this notebook, you will continue using the same small development dataset (about 11.5k rows), but the TensorFlow implementation will change to:
<ol>
  <li> Read data in batches </li>
  <li> Support shared or distributed filesystems, like Google Cloud Storage, HDFS, AWS S3</li>
  <li> Enable custom features based on individual rows from data</li>
</ol>
The Pandas function in the previous notebook also used batching during training but only after it had read the entire dataset into memory. With a large dataset, this isn't an option.



---
Before you start, **make sure that you are logged in with your student account**. Otherwise you may incur Google Cloud charges for using this notebook. 

---

In [0]:
import tensorflow as tf
import numpy as np
import shutil
import tensorflow as tf

print("Your TensorFlow version:")
print tf.__version__

#@markdown Remember to uncheck "Reset all runtimes before running"

#@markdown As you know, reseting the runtime will delete any files you may have on your notebook file system. 
#@markdown ![](https://i.imgur.com/9dgw0h0.png)

# Copy taxi-*.csv files from github if they are missing from the runtime.
!wget --quiet -nc https://github.com/osipov/training-data-analyst/raw/master/bootcamps/serverless_ml/taxi-11k-datasets.zip  
!unzip -q -n taxi-11k-datasets.zip 

<h2> 1. Refactor the input </h2>

The method `read_dataset` was implemented in an earlier lab, but now is the time to make it more general and performant. Instead of using Pandas data frames as the data source, the following implementation relies on the TensorFlow Dataset API.

In [0]:
CSV_COLUMNS = ['fare_amount', 'pickuplon','pickuplat','dropofflon','dropofflat','passengers', 'key']
LABEL_COLUMN = 'fare_amount'
DEFAULTS = [[0.0], [-74.0], [40.0], [-74.0], [40.7], [1.0], ['nokey']]

def read_dataset(filename, mode, batch_size = 512):
  def _input_fn():
    def decode_csv(value_column):
      columns = tf.decode_csv(value_column, record_defaults = DEFAULTS)
      features = dict(zip(CSV_COLUMNS, columns))
      label = features.pop(LABEL_COLUMN)
      return features, label

    # Create list of files that match pattern
    file_list = tf.gfile.Glob(filename)

    # Create dataset from file list
    dataset = tf.data.TextLineDataset(file_list).map(decode_csv)
    if mode == tf.estimator.ModeKeys.TRAIN:
        num_epochs = None # indefinitely
        dataset = dataset.shuffle(buffer_size = 10 * batch_size)
    else:
        num_epochs = 1 # end-of-input after this

    dataset = dataset.repeat(num_epochs).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()
  return _input_fn
    

def get_train():
  return read_dataset('./taxi-train.csv', mode = tf.estimator.ModeKeys.TRAIN)

def get_valid():
  return read_dataset('./taxi-valid.csv', mode = tf.estimator.ModeKeys.EVAL)

def get_test():
  return read_dataset('./taxi-test.csv', mode = tf.estimator.ModeKeys.EVAL)

<h2> 2. Refactor the way features are created. </h2>

For now, pass these through (same as previous lab).  However, refactoring this way will enable us to break the one-to-one relationship between inputs and features.

In [0]:
INPUT_COLUMNS = [
    tf.feature_column.numeric_column('pickuplon'),
    tf.feature_column.numeric_column('pickuplat'),
    tf.feature_column.numeric_column('dropofflat'),
    tf.feature_column.numeric_column('dropofflon'),
    tf.feature_column.numeric_column('passengers'),
]

def add_more_features(feats):
  # Nothing to add (yet!)
  return feats

feature_cols = add_more_features(INPUT_COLUMNS)

<h2> Create and train the model </h2>

Note that we train for num_steps * batch_size examples.

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

OUTDIR = 'taxi_trained'
shutil.rmtree(OUTDIR, ignore_errors = True) # start fresh each time

model = tf.estimator.LinearRegressor(
      feature_columns = feature_cols, model_dir = OUTDIR)

model.train(input_fn = get_train(), steps = 100);  

<h3> Evaluate model </h3>

As before, evaluate on the validation data.  We'll do the third refactoring (to move the evaluation into the training loop) in the next lab.

In [0]:
def print_rmse(model, name, input_fn):
  metrics = model.evaluate(input_fn = input_fn, steps = 1)
  print 'RMSE on {} dataset = {}'.format(name, np.sqrt(metrics['average_loss']))
  
print_rmse(model, 'validation', get_valid())

<h1>Recap</h1>
In this lab, you have confirmed that your model is still working as expected, even after you have changed the implementation to read the datasets using batches. The RMSE is still not where you want it to be but now you have prepared the code to scale to much larger datasets, including those that do not fit into the memory of a single server. Also, since the code in this notebook is using TensorFlow's `tf.gfile` library,  it can access datasets stored in cloud-based storage services like Google Cloud Storage (GCS). With GCS, you can scale to petabytes of training and read files that are up to 5 terabytes in size.

Copyright 2019 Counter Factual .AI LLC. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License